In [ ]:
using Pkg
Pkg.activate(expanduser("./"))
Pkg.update()
Pkg.precompile()

In [ ]:
using TinyMachines; tm=TinyMachines
using Flux
using CUDA
dev = CUDA.has_cuda_gpu() ? gpu : cpu

In [ ]:
# Synth data
Xs = rand(Float32, 64,64,3,10)
ys = rand(Bool, 64,64,2,10)

data = Flux.DataLoader((Xs, ys)) |> dev

In [ ]:
# model
model = ESPnet(3,2) |> dev;

In [ ]:
# loss
loss(model,x,y) = Flux.crossentropy(model(x), y, dims=3)

In [ ]:
# optimizer
opt = Flux.Adam()
opt_state = Flux.setup(opt, model)

In [ ]:
# train one epoch
Flux.train!(loss, model, data, opt_state)

In [ ]:
# inference
X = rand(Float32, 64,64,3,1) |> dev
ŷ = model(X) |> cpu
@assert size(ŷ) == (64,64,2,1)

In [ ]:
model.encoder[1]